<a href="https://colab.research.google.com/github/poojithamoganti/BERT_MLM/blob/main/BERT_MLM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Masked language Model - BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [ ]:
model_name = "bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
mask = tokenizer.mask_token

In [ ]:
text = f'I want to {mask} noodles for tonight.'

In [ ]:
tokens = tokenizer.tokenize(text)

In [ ]:
print(tokens)

['I', 'want', 'to', '[MASK]', 'no', '##odles', 'for', 'tonight', '.']


In [ ]:
input = tokenizer(text, return_tensors='pt')

In [ ]:
print(input)

{'input_ids': tensor([[  101,   146,  1328,  1106,   103,  1185, 27267,  1111,  3568,   119,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
output = model(**input)

In [ ]:
print(output)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3904,  -7.2717,  -7.4584,  ...,  -6.3108,  -5.9067,  -6.4273],
         [ -8.0264,  -8.3340,  -8.1236,  ...,  -6.8031,  -6.5655,  -7.0952],
         [-11.3938, -11.2042, -11.9486,  ...,  -8.2368,  -6.3560,  -7.6512],
         ...,
         [ -9.1182,  -9.2570,  -9.1846,  ...,  -7.3273,  -5.6608,  -8.4919],
         [-11.7496, -11.8306, -11.9147,  ...,  -9.4423,  -9.1980, -10.8806],
         [-12.6505, -12.1975, -12.9181,  ..., -10.3242,  -9.9766, -12.1598]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


For this Bert model it is specificed for masked. Here we have logits.
Logits are the raw and normalized predictions for the tokens.It is the scores that are given to the words which can better fit in the masked position.

In [ ]:
logits =output.logits.detach().numpy()[0]
print(logits)

[[ -7.3903813  -7.271694   -7.45842   ...  -6.310818   -5.906656
   -6.4272623]
 [ -8.02639    -8.33403    -8.123601  ...  -6.8030787  -6.565531
   -7.0951757]
 [-11.393796  -11.204183  -11.948609  ...  -8.236803   -6.3559904
   -7.651233 ]
 ...
 [ -9.118248   -9.257047   -9.184595  ...  -7.327305   -5.6608024
   -8.491889 ]
 [-11.749643  -11.830594  -11.9147005 ...  -9.442263   -9.19798
  -10.88065  ]
 [-12.650478  -12.197494  -12.918072  ... -10.324165   -9.976563
  -12.159829 ]]


the total len of tokens is 9 which is number of words. in len of logits we have extra two which are start and end extra tokens

In [ ]:
logits.shape

(11, 28996)

In [ ]:
len(tokens)

9

We are now considering only logits of the masked token

In [ ]:
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-5.9591274 -6.0355253 -5.7250733 ... -5.626595  -3.8903089 -4.1469383]


These predictied logits can be transformed into probabilities using softmax function

In [ ]:
confidence_score = softmax(mask_logits)
print(confidence_score)
confidence_score.sum()

[1.6584157e-09 1.5364368e-09 2.0957622e-09 ... 2.3126541e-09 1.3127141e-08
 1.0155890e-08]


1.0000002

The transformer model generates a list of scores for the words which can fill in the masked position

In [ ]:
for i in np.argsort(confidence_score)[::-1][:5]:
  pred = tokenizer.decode(i)
  score = confidence_score[i]
  print(pred,score)
  print(text.replace(mask,pred),score)

make 0.38526544
I want to make noodles for tonight. 0.38526544
eat 0.21861781
I want to eat noodles for tonight. 0.21861781
have 0.07889578
I want to have noodles for tonight. 0.07889578
get 0.050771475
I want to get noodles for tonight. 0.050771475
do 0.021016141
I want to do noodles for tonight. 0.021016141


here we got top 5 prediction of the masked words